In [29]:
import pandas as pd
import joblib   
import numpy as np
from sklearn.model_selection import train_test_split
from SimulateDay import scale_data, predict_action, get_stock_data, add_columns
import xgboost as xgb
import joblib


def stock_market_simulation(model, initial_cash, days, stock, stock_name, monthly_injection=0):
    # Add Taxes and Fees
    cash = initial_cash
    invested = cash
    shares_held = 0
    portfolio_value = []
    scaled = scale_data(stock)
    modelDecisionDf = pd.DataFrame(
        columns=['Stock Name', 'Day', 'Action', 'Cash', 'Shares Held', 'Portfolio Value'])

    days = min(days, len(stock))
    for i in range(days):
        stock_price = stock['Close'].iloc[i]
        strategy = predict_action(scaled.iloc[i].to_dict(), model)
        if i // 30 == 0:
            cash += monthly_injection
            invested += monthly_injection
        if strategy == 'Buy' and cash >= stock_price:
            # Buy one share if cash is sufficient
            cash -= stock_price
            shares_held += 1
            print(f"Day {i}: Bought 1 share at {stock_price}, Cash left: {cash}")

        elif strategy == 'Buy' and cash < stock_price:
            shares_held += cash / stock_price
            cash = 0
            print(
                f"Day {i}: Bought {cash / stock_price} shares at {stock_price}, Cash left: {cash}")

        elif strategy == 'Sell' and shares_held > 0:
            # Sell one share if we hold any
            cash += stock_price
            shares_held -= 1
            print(f"Day {i}: Sold 1 share at {stock_price}, Cash: {cash}")

        elif cash == 0:
            injection = input(
                f'Your current portfolio value is {portfolio_value[i-1]}. The stock price is {stock_price}. Enter the amount of cash you want to inject: ')
            if injection:
                cash += float(injection)
                invested += float(injection)
                print(f"Day {i}: Cash injected: {injection}, Cash: {cash}")
                if cash < stock_price:
                    break

        # Calculate the total portfolio value (cash + stock holdings)
        portfolio_value_at_time = cash + (shares_held * stock_price)
        portfolio_value.append(portfolio_value_at_time)

        new_row = pd.DataFrame({
            'Stock Name': [stock_name],
            'Day': [i],
            'Date': [stock['Date'].iloc[i]],
            'Action': [strategy],
            'Stock Price': [stock_price],
            'Cash': [cash],
            'Shares Held': [shares_held],
            'Portfolio Value': [portfolio_value_at_time]
        })
        modelDecisionDf = pd.concat(
            [modelDecisionDf, new_row], ignore_index=True)

    # Final results
    final_portfolio_value = cash + (shares_held * stock['Close'].iloc[-1])
    print(f'Total cash invested: {invested}')
    print(f'Stock {stock["Symbol"].iloc[0]}')
    print(f"Final Portfolio Value: {final_portfolio_value}")
    print(f"Cash: {cash}, Shares held: {shares_held}")
    return modelDecisionDf, cash
    # return portfolio_value, final_portfolio_value


In [94]:
def train_Optimal_Action(symbol,action_column):
    features = ['Volume', 'MA_10', 'MA_20', 'MA_50', 'MA_200', 'std_10',
                'std_20', 'std_50', 'std_200', 'upper_band_10', 'lower_band_10',
                'upper_band_20', 'lower_band_20', 'upper_band_50', 'lower_band_50',
                'upper_band_200', 'lower_band_200', 'Golden_Cross_Short', 'Golden_Cross_Medium',
                'Golden_Cross_Long', 'Death_Cross_Short', 'Death_Cross_Medium', 'Death_Cross_Long',
                'ROC', 'AVG_Volume_10', 'AVG_Volume_20', 'AVG_Volume_50', 'AVG_Volume_200', 'Doji',
                'Bullish_Engulfing', 'Bearish_Engulfing', 'MACD', 'Signal', 'MACD_Hist', 'TR', 'ATR',
                'RSI_10_Day', '10_Day_ROC', 'Resistance_10_Day', 'Support_10_Day', 'Resistance_20_Day',
                'Support_20_Day', 'Resistance_50_Day', 'Support_50_Day', 'Volume_MA_10', 'Volume_MA_20',
                'Volume_MA_50', 'OBV', 'Z-score']
    print('Loading data for', symbol, '...')
    stock_df = get_stock_data(symbol)
    print(f'Adding columns for {symbol}...')
    stock_df = add_columns(stock_df)
    preprocessed = scale_data(stock_df)
    X = preprocessed[features]
    y = stock_df['Action']
    # y = y.map({'Buy': 0, 'Sell': 1, 'Hold': 2})
    X_train, _, y_train, _ = train_test_split(
        X, y, test_size=0.3, random_state=42)
    print(f'Training model for {symbol}...')
    model = xgb.XGBClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print(f'Saving model for {symbol}...')
    joblib.dump(model, 'OptimalActionModel.pkl')


In [117]:
train_Optimal_Action('AMD', 'Action')

Loading data for AMD ...
Adding columns for AMD...


/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:206: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  '10_Day_Return', '20_Day_Return', '50_Day_Return', '200_Day_Return']].idxmax(axis=1)
/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:206: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  '10_Day_Return', '20_Day_Return', '50_Day_Return', '200_Day_Return']].idxmax(axis=1)


Training model for AMD...
Saving model for AMD...


In [56]:

specific_model_decisions = pd.read_csv('data/specific_model_decisions.csv')
general_model_decisions = pd.read_csv('data/general_model_decisions.csv')
general_model_decisions = general_model_decisions.sort_values(by='Stock Name')
specific_model_decisions = specific_model_decisions.sort_values(by='Stock Name')

general_model = xgb.Booster()
general_model.load_model('models/all_stocks_incremental_model.pkl') 
specific_model_decisions['Date'] = pd.to_datetime(specific_model_decisions['Date'], format='%Y-%m-%d')
specific_model_decisions.loc[specific_model_decisions['Date'] == '2024-10-10', 'Day'] = 1
general_model_decisions.loc[general_model_decisions['Date'] == '2024-10-10', 'Day'] = 1
specific_model_decisions['Action'].value_counts()
# general_model_decisions.to_csv('data/general_model_decisions.csv', index=False)
# specific_model_decisions.to_csv('data/specific_model_decisions.csv', index=False)


Action
Sell    512
Buy     288
Hold    206
Name: count, dtype: int64

In [174]:
import pandas as pd

# Load and sort the data by stock name
test_decisions = pd.read_csv('data/specific_model_decisions.csv').sort_values(by='Stock Name')

# Set the initial Portfolio Value for Day 0 to 10000
test_decisions.loc[test_decisions['Day'] == 0, 'Portfolio Value'] = 10000

# Adjust the Cash column based on shares purchased and stock price
test_decisions['Cash'] = test_decisions['Cash'] + (test_decisions['Shares Held'] * test_decisions['Stock Price'])

# Update the Portfolio Value for non-Day 0 rows
test_decisions.loc[test_decisions['Day'] != 0, 'Portfolio Value'] = (
    test_decisions['Cash'] + (test_decisions['Shares Held'] * test_decisions['Stock Price'])
)

# Optional: Save the adjusted dataframe to a CSV file
# test_decisions.to_csv('data/specific_model_decisions.csv', index=False)

# For testing purposes, filter rows where portfolio value is less than 10000
test_decisions = test_decisions[test_decisions["Day"] == 0]
test_decisions.to_csv('data/specific_model_decisions.csv', index=False)

In [58]:
buy_specific = specific_model_decisions[specific_model_decisions['Action'] == 'Buy']
buy_specific

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
12,ACN,0,Buy,365.140015,9634.859985,1.0,10000.000000,2024-09-10
13,ACN,1,Buy,359.799988,9275.059998,1.0,9634.859985,2024-10-10
26,AEP,0,Buy,97.519997,9902.480003,1.0,10000.000000,2024-09-10
27,AEP,1,Buy,97.389999,9805.090004,1.0,9902.480003,2024-10-10
31,AFL,1,Buy,111.529999,9777.965004,1.0,9889.495003,2024-10-10
...,...,...,...,...,...,...,...,...
983,WRB,1,Buy,57.919998,9884.110001,1.0,9942.029999,2024-10-10
994,XOM,1,Buy,123.559998,9754.290001,1.0,9877.849998,2024-10-10
995,XOM,0,Buy,122.150002,9877.849998,1.0,10000.000000,2024-09-10
1000,ZBH,0,Buy,103.000000,9897.000000,1.0,10000.000000,2024-09-10


In [118]:
stock_data = get_stock_data('AMD')
stock_data = stock_data[(stock_data['Date'] >= '2021-01-01') & (stock_data['Date'] <= '2022-01-01')]
Optimal_Action_model = joblib.load('OptimalActionModel.pkl')
specific_model0 = joblib.load('models/AMD_model.pkl')
specific_decisions, _ = stock_market_simulation(specific_model0, 10000, 365, stock_data, 'AMD', 0)

/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:206: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  '10_Day_Return', '20_Day_Return', '50_Day_Return', '200_Day_Return']].idxmax(axis=1)
/var/folders/75/l95mxw1j5w9ddljb82_6_m_40000gn/T/ipykernel_5125/413777460.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modelDecisionDf = pd.concat(


Day 1: Bought 1 share at 92.7699966430664, Cash left: 9907.230003356934
Day 2: Sold 1 share at 90.33000183105467, Cash: 9997.560005187988
Day 4: Bought 1 share at 94.58000183105467, Cash left: 9902.980003356934
Day 6: Bought 1 share at 95.36000061035156, Cash left: 9807.620002746582
Day 7: Bought 1 share at 91.77999877929688, Cash left: 9715.840003967285
Day 8: Sold 1 share at 90.79000091552734, Cash: 9806.630004882812
Day 9: Sold 1 share at 88.20999908447266, Cash: 9894.840003967285
Day 10: Sold 1 share at 89.44999694824219, Cash: 9984.290000915527
Day 25: Bought 1 share at 90.91000366210938, Cash left: 9893.379997253418
Day 27: Bought 1 share at 92.66000366210938, Cash left: 9800.719993591309
Day 28: Bought 1 share at 93.7699966430664, Cash left: 9706.949996948242
Day 29: Bought 1 share at 91.45999908447266, Cash left: 9615.48999786377
Day 30: Bought 1 share at 89.94000244140625, Cash left: 9525.549995422363
Day 31: Bought 1 share at 88.63999938964844, Cash left: 9436.909996032715
Da

In [119]:
Optimal_Action_decision,_ = stock_market_simulation(Optimal_Action_model, 10000, 365, stock_data, 'AMD',0)

/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:206: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  '10_Day_Return', '20_Day_Return', '50_Day_Return', '200_Day_Return']].idxmax(axis=1)
/var/folders/75/l95mxw1j5w9ddljb82_6_m_40000gn/T/ipykernel_5125/413777460.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modelDecisionDf = pd.concat(


Day 2: Bought 1 share at 90.33000183105467, Cash left: 9909.669998168945
Day 6: Bought 1 share at 95.36000061035156, Cash left: 9814.309997558594
Day 7: Bought 1 share at 91.77999877929688, Cash left: 9722.529998779297
Day 8: Bought 1 share at 90.79000091552734, Cash left: 9631.73999786377
Day 9: Sold 1 share at 88.20999908447266, Cash: 9719.949996948242
Day 16: Bought 1 share at 88.83999633789062, Cash left: 9631.110000610352
Day 17: Sold 1 share at 87.5199966430664, Cash: 9718.629997253418
Day 18: Sold 1 share at 85.63999938964844, Cash: 9804.269996643066
Day 21: Sold 1 share at 87.88999938964844, Cash: 9892.159996032715
Day 29: Bought 1 share at 91.45999908447266, Cash left: 9800.699996948242
Day 30: Bought 1 share at 89.94000244140625, Cash left: 9710.759994506836
Day 31: Bought 1 share at 88.63999938964844, Cash left: 9622.119995117188
Day 33: Sold 1 share at 85.37000274658203, Cash: 9707.48999786377
Day 36: Sold 1 share at 82.41999816894531, Cash: 9789.909996032715
Day 39: Sold 1

In [120]:
general_model_decisions, _ = stock_market_simulation(general_model, 10000, 365, stock_data, 'AMD',0)

/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:206: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  '10_Day_Return', '20_Day_Return', '50_Day_Return', '200_Day_Return']].idxmax(axis=1)


Day 0: Bought 1 share at 92.3000030517578, Cash left: 9907.699996948242
Day 1: Bought 1 share at 92.7699966430664, Cash left: 9814.930000305176
Day 2: Bought 1 share at 90.33000183105467, Cash left: 9724.599998474121
Day 4: Bought 1 share at 94.58000183105467, Cash left: 9630.019996643066
Day 5: Bought 1 share at 97.25, Cash left: 9532.769996643066
Day 6: Bought 1 share at 95.36000061035156, Cash left: 9437.409996032715
Day 7: Bought 1 share at 91.77999877929688, Cash left: 9345.629997253418
Day 8: Sold 1 share at 90.79000091552734, Cash: 9436.419998168945
Day 9: Sold 1 share at 88.20999908447266, Cash: 9524.629997253418
Day 10: Sold 1 share at 89.44999694824219, Cash: 9614.07999420166
Day 11: Sold 1 share at 88.75, Cash: 9702.82999420166
Day 12: Sold 1 share at 91.52999877929688, Cash: 9794.359992980957


/var/folders/75/l95mxw1j5w9ddljb82_6_m_40000gn/T/ipykernel_5125/413777460.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modelDecisionDf = pd.concat(


Day 13: Sold 1 share at 92.79000091552734, Cash: 9887.149993896484
Day 14: Bought 1 share at 94.12999725341795, Cash left: 9793.019996643066
Day 15: Sold 1 share at 94.70999908447266, Cash: 9887.729995727539
Day 16: Sold 1 share at 88.83999633789062, Cash: 9976.56999206543
Day 26: Bought 1 share at 92.3499984741211, Cash left: 9884.219993591309
Day 27: Bought 1 share at 92.66000366210938, Cash left: 9791.5599899292
Day 28: Bought 1 share at 93.7699966430664, Cash left: 9697.789993286133
Day 29: Bought 1 share at 91.45999908447266, Cash left: 9606.32999420166
Day 30: Bought 1 share at 89.94000244140625, Cash left: 9516.389991760254
Day 31: Bought 1 share at 88.63999938964844, Cash left: 9427.749992370605
Day 32: Bought 1 share at 89.58000183105469, Cash left: 9338.16999053955
Day 33: Bought 1 share at 85.37000274658203, Cash left: 9252.799987792969
Day 34: Sold 1 share at 84.73999786376953, Cash: 9337.539985656738
Day 35: Sold 1 share at 86.94000244140625, Cash: 9424.479988098145
Day 36

In [68]:
specific_decisions.tail(10)

,Stock Name,Day,Action,Cash,Shares Held,Portfolio Value,Date,Stock Price
918,AAPL,918,Buy,6877.499878,191,50114.168945,2024-09-25,226.369995
919,AAPL,919,Buy,6649.979874,192,50333.820694,2024-09-26,227.520004
920,AAPL,920,Buy,6422.18988,193,50385.658585,2024-09-27,227.789993
921,AAPL,921,Buy,6189.18988,194,51391.189880,2024-09-30,233.000000
922,AAPL,922,Buy,5962.979874,195,50073.931183,2024-10-01,226.210007
923,AAPL,923,Buy,5736.199875,196,50185.079636,2024-10-02,226.779999
924,AAPL,924,Buy,5510.529877,197,49967.519516,2024-10-03,225.669998
925,AAPL,925,Buy,5283.729874,198,50190.130478,2024-10-04,226.800003
926,AAPL,926,Sell,5505.419876,197,49178.350357,2024-10-07,221.690002
927,AAPL,927,Hold,5505.419876,197,50626.298553,2024-10-10,229.039993


In [69]:
Optimal_Action_decision.tail(10)

,Stock Name,Day,Action,Cash,Shares Held,Portfolio Value,Date,Stock Price
918,GOOG,918,Buy,39701.520004,2,40154.259995,2024-09-25,226.369995
919,GOOG,919,Sell,39929.040009,1,40156.560013,2024-09-26,227.520004
920,GOOG,920,Buy,39701.250015,2,40156.830002,2024-09-27,227.789993
921,GOOG,921,Hold,39701.250015,2,40167.250015,2024-09-30,233.000000
922,GOOG,922,Hold,39701.250015,2,40153.670029,2024-10-01,226.210007
923,GOOG,923,Hold,39701.250015,2,40154.810013,2024-10-02,226.779999
924,GOOG,924,Hold,39701.250015,2,40152.590012,2024-10-03,225.669998
925,GOOG,925,Hold,39701.250015,2,40154.850021,2024-10-04,226.800003
926,GOOG,926,Hold,39701.250015,2,40144.630020,2024-10-07,221.690002
927,GOOG,927,Hold,39701.250015,2,40159.330002,2024-10-10,229.039993


In [70]:
general_model_decisions.tail(10)

,Stock Name,Day,Action,Cash,Shares Held,Portfolio Value,Date,Stock Price
918,GOOG,918,Sell,40282.160240,0,40282.160240,2024-09-25,226.369995
919,GOOG,919,Sell,40282.160240,0,40282.160240,2024-09-26,227.520004
920,GOOG,920,Sell,40282.160240,0,40282.160240,2024-09-27,227.789993
921,GOOG,921,Hold,40282.160240,0,40282.160240,2024-09-30,233.000000
922,GOOG,922,Buy,40055.950233,1,40282.160240,2024-10-01,226.210007
923,GOOG,923,Sell,40282.730232,0,40282.730232,2024-10-02,226.779999
924,GOOG,924,Sell,40282.730232,0,40282.730232,2024-10-03,225.669998
925,GOOG,925,Sell,40282.730232,0,40282.730232,2024-10-04,226.800003
926,GOOG,926,Sell,40282.730232,0,40282.730232,2024-10-07,221.690002
927,GOOG,927,Hold,40282.730232,0,40282.730232,2024-10-10,229.039993


1000 Day Test Start Date:2021-01-30
    SpecificModel: 1 | OptimalActionModel: 0 | GeneralModel: 0